# Проектная работа 1. Предобработка данных

# Исследование надёжности заёмщиков

**Цель:** Получить практичесские навыки в предобработке данных: обрабатывать пропущенные данные, работать с типами данных, выделять леммы, категоризировать данные.  
  
**План выполнения работы:**  
- 1. Изучение общей информации о данных  
- 2. Предобработка данных   
    - 2.1 Заполнение пропущенных данных  
    - 2.2 Замена типа данных  
    - 2.3 Удаление дубликатов  
    - 2.4 Лемматизация целей получения кредитов  
    - 2.5 Категоризация данных    
- 3. Исследование данных  
    - 3.1 Зависимость между наличием детей и возвратом кредита в срок  
    - 3.2 Зависимость между семейным положением и возвратом кредита в срок  
    - 3.3 Зависимость между уровнем дохода и возвратом кредита в срок  
    - 3.4 Влияние целей кредита на его возврат в срок  
- 4. Общий вывод  
  
  
**Описание Данных:**  
Данные представляют собой таблицу со статистическиой информацией о платёжеспособности клиентов:
- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

## Шаг 1. Изучение общей информации о данных

In [1]:
# подключаем библиотеку pandas для работы с таблицами и библиотеку display для вывода табличных данных
import pandas as pd
from IPython.display import display
# импортируем лемматизатор Mystem из библиотеки pymystem3
from pymystem3 import Mystem

In [2]:
# читаем данные из .csv файла с помощью метода read_csv()
customer_solvency = pd.read_csv('/datasets/data.csv')
# выведем первые 5 строк таблицы с помощью метода head()
display(customer_solvency.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
# с помощью метода info() изучим структуру таблицы: типы данных, колличество строк, столбцов, 
# пропущенных данных.
customer_solvency.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Данные успешно загружены и корректно записаны в DataFrame. Таблица состоит из 12 столбцов и 21525 записей. Присутствуют пропущенные значения, исследования которых будут проведены далее.

<a id='секция2'></a>
## Шаг 2. Предобработка данных

<a id='секция2.1'></a>
### 2.1 Заполнение пропущенных данных

Таблица данных состоит из 21525 строк и 12 столбцов. Данные пропущены в столбцах `days_employed` (общий трудовой стаж в днях) и `total_income` (ежемесячный доход). Причём колличество пропущенных данных одинаково. Возможно, в этом есть закономерность.

In [4]:
# сохраним в переменной missed_days_employed все строки с пропущенным значением 'days_employed'
missed_days_employed = customer_solvency.loc[customer_solvency['days_employed'].isnull()]
print('Колличество строк с пропущенным трудовым стажем:', len(missed_days_employed))
# проверим, что для всех этих строк значения в 'total_income' также пропущены
display(missed_days_employed.loc[missed_days_employed['total_income'].isnull() == False])

Колличество строк с пропущенным трудовым стажем: 2174


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Около 10% строк имеют пропуски в данных. Их нельзя просто удалить без ущерба статистике.

#### Исследование некорректных данных в столбце `days_employed`:

Значения в столбцах `days_employed` и `total_income` пропущены в одних и тех же строках. Возможно, произошёл сбой системы при выгрузке или записи данных. На это также указывают отрицательные значения в `days_employed`.
Так же вероятно были ошибки при заполнении данных: Общий трудовой стаж в 340266 дней выглядит неправдоподобным при возрасте клиента 53 года. Трудовой стаж человека не может быть больше 30000 дней (грубая оценка: за 75-80 лет работы). Найдем колличество записей, превышающих это значение.

In [5]:
# с помощью метода loc[] и логической операции отберем записи с 'days_employed' > 30000 и
# выведем колличество записей с помощью функции len()
wrong_days_employed = len(customer_solvency.loc[customer_solvency['days_employed'] > 30000])
print('Колличество строк с неверным трудовым стажем:', wrong_days_employed)
# аналогично найдем колличество строк с отрицательным тррудовым стажем.
neg_days_employed = len(customer_solvency.loc[customer_solvency['days_employed'] < 0])
print('Колличество строк с отрицательным трудовым стажем:', wrong_days_employed)

Колличество строк с неверным трудовым стажем: 3445
Колличество строк с отрицательным трудовым стажем: 3445


- Почти половина данных в `total_income` имеет некорректное или пропущенное значение. Поскольку причины неизвестны, изменение некорректных данных может только усугубить ситуацию.
- Данные могли быть введены в часах (а не в днях), что объяснило бы высокие значения. Так же системный сбой мог изменить формат данных (точка отсчета дней могла измениться, что привело к отрицательным значениям).
- В любом случае необходимо дальнейшее исследование проблемы с сотрудниками банка.
- На данном этапе заменим пропущенные данные в `days_employed` нулями для наглядности.

In [6]:
# с помощью метода isnull() найдём все строки с пропущенным значением в 'days_employed' и заменим
# их на 0
customer_solvency['days_employed'] = customer_solvency['days_employed'].fillna(0)
display(customer_solvency.loc[customer_solvency['days_employed'] == 0].head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,0.0,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,0.0,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,0.0,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,0.0,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,0.0,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,0.0,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,0.0,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,0.0,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,0.0,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,0.0,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


- Рассмотрим столбец `total_income`.
Сгруппируем данные по уровню образования и выведем статистические данные уровня доходов для каждой группы (минимальное, максимальное значения, среднее арифметические и медиану)

In [7]:
# методом groupby() сгруппируем значения по 'education_id'.
# методом agg() применим статистические функции к столбцу 'total_income'
customer_solvency_stat = customer_solvency.groupby('education_id').agg({'total_income':['min','max','mean','median']})
display(customer_solvency_stat)

total_income                                            
                       min           max           mean         median
education_id                                                          
0             32178.213678  2.265604e+06  207142.515219  175340.818855
1             20667.263793  1.726276e+06  153715.643971  136478.643244
2             34466.133539  9.584346e+05  181534.022774  160115.398644
3             25308.586849  4.900673e+05  132155.513626  117137.352825
4             98752.495442  2.684112e+05  174750.155792  157259.898555

Минимальные и максимальные значения месячного дохода отличаются от средних примерно в 10 раз. Также средние значения достаточно сильно отличаются от медианных. В данном случае заполним пропуски медианами.

In [8]:
# для замены пропущенных значений используем метод fillna()
# в аргументе метода сгруппируем данные по 'education_id' и для каждой группы посчитаем медианное 
# значение 'total_income'
customer_solvency['total_income'] = customer_solvency['total_income'].fillna(customer_solvency.groupby('education_id')['total_income'].transform('median'))

- Рассмотрим столбец `gender`:

In [9]:
# найдем колличество вхождений каждого значения с помощью метода value_counts()
display(customer_solvency['gender'].value_counts())
display(customer_solvency.loc[customer_solvency['gender'] == 'XNA'])

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Некорректное значение `XNA` встречается 1 раз. По остальным значениям в данной записи невозможно 
определить пол клиента. Одну запись можно удалить. Это не повлияет на статистику.

In [10]:
customer_solvency = customer_solvency.loc[customer_solvency['gender'] != 'XNA']

### Вывод

Около 10% записей имеют пропущенные значения. В данных, не участвующих в дальнейших исследованиях, пропуски были заменены нулями, в столбце `total_income` данные были заменены медианными значениями по группам с соответствующим уровнем образования клиента. Одиночные строки с некорректными данными были удалены.  
В столбце `days_employed` около 50% данных пропущены или некорректны. Необходимо дальнейшее исследование причин с сотрудниками банка.

<a id='секция2.2'></a>
### 2.2 Замена типа данных

- Колличество трудовых дней может быть только целым числом. Так же сделаем ежемесячный доход целым числом для наглядности. 
- Преобразуем типы данных в столбцах `days_employed` и `total_income` c `float64` на целочисленный `int64`.
- Преобразование проведем методом `astype()`. Он произведет изменение типа для вещественных чисел и выдаст ошибку, если данные изначально были введены некорректно (например, если встретятся данные в виде строк вместо вещественных чисел)

In [11]:
# типы данных преобразуем методом astype().
# с помощью try-except отслеживаем некорректные типы данных
try:
    customer_solvency['days_employed'] = customer_solvency['days_employed'].astype('int64')
    customer_solvency['total_income'] = customer_solvency['total_income'].astype('int64')
except:
    print('Некорректный тип данных. Пожалуйста, убедитесь, что данные хранятся в виде вещественных чисел')

### Вывод

Вещественные данные успешно заменены на целочисленный тип `int64` применением метода `astype()`.

<a id='секция2.3'></a>
### 2.3 Удаление дубликатов

- Для поиска дубликатов сперва преобразуем все строковые типы данных к нижнему регистру.
- Затем посчитаем колличество дублированных строк и выведем их для анализа.

In [12]:
# для преобразования к нижнему регистру используем метод str.lower()
# затем запишем преобразованные данные в таблицу
customer_solvency['education'] = customer_solvency['education'].str.lower()
customer_solvency['family_status'] = customer_solvency['family_status'].str.lower()

In [13]:
print('Колличество дублированных данных: ', customer_solvency.duplicated().sum())

Колличество дублированных данных:  71


In [14]:
display(customer_solvency.loc[customer_solvency.duplicated()].head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,0,41,среднее,1,женат / замужем,0,F,сотрудник,0,136478,покупка жилья для семьи
3290,0,0,58,среднее,1,гражданский брак,1,F,пенсионер,0,136478,сыграть свадьбу
4182,1,0,34,высшее,0,гражданский брак,1,F,сотрудник,0,175340,свадьба
4851,0,0,60,среднее,1,гражданский брак,1,F,пенсионер,0,136478,свадьба
5557,0,0,58,среднее,1,гражданский брак,1,F,пенсионер,0,136478,сыграть свадьбу
6312,0,0,30,среднее,1,женат / замужем,0,M,сотрудник,0,136478,строительство жилой недвижимости
7808,0,0,57,среднее,1,гражданский брак,1,F,пенсионер,0,136478,на проведение свадьбы
7921,0,0,64,высшее,0,гражданский брак,1,F,пенсионер,0,175340,на проведение свадьбы
7938,0,0,71,среднее,1,гражданский брак,1,F,пенсионер,0,136478,на проведение свадьбы
8583,0,0,58,высшее,0,не женат / не замужем,4,F,пенсионер,0,175340,дополнительное образование


- Все дублированные данные являются строками, в которых ранее были обработаны пропущенные значения.
Поскольку в таблице нет уникальных идентификаторов клиентов, невозможно точно отверждать: данные дублированы или информация по разным клиентам совпала случайным образом.
- Возможные причины появления дубликатов: повторное заполнение или сохранение данных о клиенте в базе данных, сбой в системе при копировании/выгрузке базы данных.
- Удалим дублированные строки в учебных целях. Для этого используем метод `drop_duplicates()` и восстановим индексы после удаления строк методом `reset_index()`

In [15]:
customer_solvency = customer_solvency.drop_duplicates().reset_index(drop = True)
print('Колличество строк после удаления дубликатов:', len(customer_solvency))

Колличество строк после удаления дубликатов: 21453


### Вывод

Былы обнаружены и успешно удалены 71 дубликат.  
Возможные причины появления дубликатов: повторное заполнение или сохранение данных о клиенте в базе данных, сбой в системе при копировании/выгрузке базы данных.

<a id='секция2.4'></a>
### 2.4 Лемматизация целей получения кредитов

Выведем все возможные цели получения кредитов для анализа. Используем метод `value_counts()`.

In [16]:
# в таблице purposes будем хранить исходные цели для кредитов и созданные после лемматизации категории
purposes = pd.DataFrame(columns = ['purpose', 'purpose_category'])
# найдем все цели в таблице data с помощью метода unique() и запишем их в новую таблицу
purposes['purpose'] = customer_solvency['purpose'].unique()
display(purposes)

,purpose,purpose_category
0,покупка жилья,NaN
1,приобретение автомобиля,NaN
2,дополнительное образование,NaN
3,сыграть свадьбу,NaN
4,операции с жильем,NaN
5,образование,NaN
6,на проведение свадьбы,NaN
7,покупка жилья для семьи,NaN
8,покупка недвижимости,NaN
9,покупка коммерческой недвижимости,NaN


Лемматизацию будем проводить с помощью библиотеки `pymystem3`. Преобразуем каждую строку в столбце `purpose` в список лемм и разделим их на 6 групп:  
**1) свадьба:** найдено несколько описаний целей, характеризующих необходимость кредита для проведения свадьбы. Сумма и срок кредита могут быть сравнительно небольшими и сильно отличаться от кредитов на другие цели. Следоватнельно, статистика по задолженностям может отличаться. По этой причине создаем отдельную цель.  
**2) автомобиль:** в данную категорию вынесены кредиты на проведение различных операций с автомобилем (покупка, обмен, иные сделки). Аналогично с целью 'свадьба' сумма и сроки кредита могут быть сравнительно небольшими. Но обязанности погашения задолженности могут ложиться как на семью (для семейного автомобиля), так и на одного человека. Следовательно, статистика по задолженностям может отличаться. Для этого создаём отдельную группу.  
**3) жилье/недвижимость:** в таблице данных выделено множество целей, связанных с недвижимостью. Причём одни цели могут относиться к частному жилью, другие к комерческой недвижимости. Частные и юридические лица могут сильно отличаться друг от друга. По данной таблице сложно определить эти две подкатегории. Необходимо обратиться к специалистам банка для более точного разделения целей и клиентов на категории. Однако в основном кредиты, связанные с недвижимостью, предполагают большие суммы заёма и долгие сроки. Как частные, так и юридические лица относятся серьёзно к таким кредитам. Поэтому статистика по задолженностям может быть схожей среди описанных подкатегорий и отличаться от остальных групп. Для всех кредитов, связанных с недвижимостью, выделяем одну категорию.  
**4) образование:** кредит на образование могут брать как взрослые клиенты со стабильным доходом, так и студенты, не имеющие стабильный доход, но получившие грант на образование. Такие кредиты могут браться на сравнительно долгий срок (срок обучения). Дата начала возврата денежных средств может быть отложена на несколько лет до получения диплома. Ввиду особых условий данной цели кредита выделяем отдельную группу.  
**5) ремонт:** с одной стороны данная цель коррелирует с целью 'жилье/недвижимость', но сумма и срок кредита в данном случае обычно гораздо меньше. Значит и результаты по задолженностям могут отличаться. Поэтому выносим 'ремонт' в отдельную категорию.   
**6) другое:** для возможных несовпадений с основными целями  

In [17]:
# инициируем Mystem() и запишем в переменнную m
m = Mystem()
def lemm_categorize(purpose):
    """
    input: purpose (object) - строка, описывает цель для кредита
    output: (object) - строка, название категории для цели кредита
    Функция принимает на вход цель кредита, лемматизирует ее и по встречающимся в цели леммам
    определяет категорию цели:
    
    Лемма:       Категория:
    свадьба      свадьба
    ремонт       ремонт
    автомобиль   автомобиль
    образование  образование
    жилье        жилье/недвижимость
    недвижимость жилье/недвижимость
    
    При несовпадении лемм ни с одной из категорий присваивается категория 'другое'
    """
    lemmas = m.lemmatize(purpose)
    if 'свадьба' in lemmas:
        return 'свадьба'
    elif 'ремонт' in lemmas:
        return 'ремонт'
    elif 'автомобиль' in lemmas:
        return 'автомобиль'
    elif 'образование' in lemmas:
        return 'образование'
    elif 'жилье' in lemmas or 'недвижимость' in lemmas:
        return 'жилье/недвижимость'
    else:
        return 'другое'

In [18]:
# для каждой цели в таблице purposes вызывается функция lemm_categorize с помощью метода apply().
# возвращенные функцией категории целей записываются в столбец 'purpose_category'
purposes['purpose_category'] = purposes['purpose'].apply(lemm_categorize)

In [19]:
display(purposes.head(10))

,purpose,purpose_category
0,покупка жилья,жилье/недвижимость
1,приобретение автомобиля,автомобиль
2,дополнительное образование,образование
3,сыграть свадьбу,свадьба
4,операции с жильем,жилье/недвижимость
5,образование,образование
6,на проведение свадьбы,свадьба
7,покупка жилья для семьи,жилье/недвижимость
8,покупка недвижимости,жилье/недвижимость
9,покупка коммерческой недвижимости,жилье/недвижимость


Категории целей кредитов успешно присвоены к каждой цели. С помощью метода `merge()` добавим `purpose_category` к основной таблице `data`:

In [20]:
customer_solvency = customer_solvency.merge(purposes, on='purpose', how='left')
display(customer_solvency.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье/недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье/недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,жилье/недвижимость
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,жилье/недвижимость
7,0,-152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,-6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,жилье/недвижимость


### Вывод

Среди целей на получение кредита было выделено 5 основных групп: `свадьба`, `ремонт`, `автомобиль`, `образование`, `жилье/недвижимость`.  
Для целей, не подходящих ни под одну категорию была выделена группа `другое`. (для возможных новых целей при дальнейшем накапливании данных).  
Лемматизация целей проводилась с использованием библиотеки `pymystem3`.

<a id='секция2.5'></a>
### 2.5 Категоризация данных

1. В предыдущем шаге был создан словарь для целей кредита. На данном шаге заменим столбец `purpose` в основной таблице на идентификатор `purpose_id`, а описание каждой цели будет храниться только в словаре. Так основная таблица будет занимать меньше места и станет более наглядной.
2. Также уберем из основной таблицы семейное положение `family_status` и уровень образования клиента `education`, оставив только идентификаторы `education_id` и `family_status_id`. Удаленные из основной таблицы данные будут храниться в словарях `education_dict` и `family_status_dict` соответственно. Так основная таблица станет более наглядна. Названия для отчетов можно будет позже брать из словарей.
3. Категоризируем данные о колличестве детей в семье. Разделим данные на категории:
    - 0: Нет детей
    - 1: Один ребенок
    - 2: Двое детей
    - 3: Многодетная семья
    - 4: Статус неизвестен (значение -1 в столбце `children`)  
Таким образом можно будет исследовать влияние семейного положения на кредитную историю клиентов.
4. Категоризируем данные о ежемесячном доходе. Разделим клиентов на 6 равных категорий по уровню доходов (примерно по 3500 клиентов в каждой категории). Одинаковые размеры выборки в каждой категории обеспечат более надёжные статистические результаты исследования.  
Таким образом клиентов можно будет сгруппировать по уровню дохода и исследовать кредитные истории отдельных групп.

In [21]:
# создадим в таблице purposes столбцы purpose_id и purpose_cat_id
purposes['purpose_id'] = purposes.index

def set_purpose_cat_id(purpose_category):
    """
    input: purpose_category (object) - строка, описывает категорию цели для кредита
    output: (int64) - целое число, идентификатор категории цели кредита
    Функция принимает на вход категорию цели кредита, и сопоставляет ей уникальное 
    десятичное целое число (идентификатор):
    
    Категория:          Идентификатор:
    свадьба             0
    ремонт              1
    автомобиль          2
    образование         3 
    жилье/недвижимость  4
    неопределённая кат. 5
    """
    
    if purpose_category == 'свадьба':
        return 0
    elif purpose_category == 'ремонт':
        return 1
    elif purpose_category == 'автомобиль':
        return 2
    elif purpose_category == 'образование':
        return 3
    elif purpose_category == 'жилье/недвижимость':
        return 4
    else:
        return 5

# с помощью метода apply применяем функцию set_purpose_cat_id ко всем 'purpose_category' 
# в таблице 'purposes'
purposes['purpose_cat_id'] = purposes['purpose_category'].apply(set_purpose_cat_id)
display(purposes.head(10))

,purpose,purpose_category,purpose_id,purpose_cat_id
0,покупка жилья,жилье/недвижимость,0,4
1,приобретение автомобиля,автомобиль,1,2
2,дополнительное образование,образование,2,3
3,сыграть свадьбу,свадьба,3,0
4,операции с жильем,жилье/недвижимость,4,4
5,образование,образование,5,3
6,на проведение свадьбы,свадьба,6,0
7,покупка жилья для семьи,жилье/недвижимость,7,4
8,покупка недвижимости,жилье/недвижимость,8,4
9,покупка коммерческой недвижимости,жилье/недвижимость,9,4


In [22]:
# с помощью метода merge добавим 'purpose_id' и 'purpose_cat_id' в основную таблицу
# и удалим из нее столбцы 'purpose' и 'purpose_category' методом drop().
customer_solvency = customer_solvency.merge(purposes, on=['purpose', 'purpose_category'], how='left')
customer_solvency = customer_solvency.drop(['purpose', 'purpose_category'], axis = 1)
display(customer_solvency.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose_id,purpose_cat_id
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,0,4
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,1,2
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,0,4
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,2,3
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,3,0
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,0,4
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,4,4
7,0,-152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,5,3
8,2,-6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,6,0
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,7,4


In [23]:
# создадим словари для уровня образования и семейного статуса и удалим столбцы из основной таблицы.
# выведем столбцы 'education', 'education_id', уберем дубликаты и сбросим индексы.
# результат сохраним в таблице education_dict
education_dict = customer_solvency[['education', 'education_id']].drop_duplicates().reset_index(drop=True)
display(education_dict)

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [24]:
# аналогично создадим словарь для семейного положения
family_status_dict = customer_solvency[['family_status', 'family_status_id']].drop_duplicates().reset_index(drop=True)
display(family_status_dict)

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


In [25]:
# Названия уровня образования и семейного положения сохранены в словарях. 
# Теперь можно удалить их из основной таблицы. Воспользуемся методом drop()
customer_solvency = customer_solvency.drop(['education', 'family_status'], axis = 1)
display(customer_solvency.head(10))

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose_id,purpose_cat_id
0,1,-8437,42,0,0,F,сотрудник,0,253875,0,4
1,1,-4024,36,1,0,F,сотрудник,0,112080,1,2
2,0,-5623,33,1,0,M,сотрудник,0,145885,0,4
3,3,-4124,32,1,0,M,сотрудник,0,267628,2,3
4,0,340266,53,1,1,F,пенсионер,0,158616,3,0
5,0,-926,27,0,1,M,компаньон,0,255763,0,4
6,0,-2879,43,0,0,F,компаньон,0,240525,4,4
7,0,-152,50,1,0,M,сотрудник,0,135823,5,3
8,2,-6929,35,0,1,F,сотрудник,0,95856,6,0
9,0,-2188,41,1,0,M,сотрудник,0,144425,7,4


In [26]:
# Категоризируем колличество детей в семье.
# создадим словарь для категорий колличества детей в семье children_dict
children_data = {'children_cat_id': [0, 1, 2, 3, 4],
                 'children_cat_name': ['Нет детей', 'Один ребенок', 'Двое детей', 
                                       'Многодетная семья', 'Статус неизвестен']}
children_columns = ['children_cat_id', 'children_cat_name']
children_dict = pd.DataFrame(data = children_data, columns = children_columns)
display(children_dict)

,children_cat_id,children_cat_name
0,0,Нет детей
1,1,Один ребенок
2,2,Двое детей
3,3,Многодетная семья
4,4,Статус неизвестен


In [27]:
def set_children_cat_id(children):
    """
    input: children (int64) - целое число, описывает колличество детей в семье
    output: (int64) - целое число, идентификатор категории колличества детей в семье
    Функция сопоставляет колличеству детей в семье одну из категорий:
    
    Children:  ID:   Категория:
    0          0     Нет детей
    1          1     Один ребенок
    2          2     Двое детей
    >2         3     Многодетная семья
    <0         4     Статус неизвестен (значение -1 в столбце 'children')
    """
    
    if children == 0:
        return 0
    elif children == 1:
        return 1
    elif children == 2:
        return 2
    elif children > 2:
        return 3
    else:
        return 4

# с помощью метода apply применяем функцию set_children_cat_id ко всем 'children' 
# в таблице 'customer_solvency'
customer_solvency['children_cat_id'] = customer_solvency['children'].apply(set_children_cat_id)
display(customer_solvency.head(10))

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose_id,purpose_cat_id,children_cat_id
0,1,-8437,42,0,0,F,сотрудник,0,253875,0,4,1
1,1,-4024,36,1,0,F,сотрудник,0,112080,1,2,1
2,0,-5623,33,1,0,M,сотрудник,0,145885,0,4,0
3,3,-4124,32,1,0,M,сотрудник,0,267628,2,3,3
4,0,340266,53,1,1,F,пенсионер,0,158616,3,0,0
5,0,-926,27,0,1,M,компаньон,0,255763,0,4,0
6,0,-2879,43,0,0,F,компаньон,0,240525,4,4,0
7,0,-152,50,1,0,M,сотрудник,0,135823,5,3,0
8,2,-6929,35,0,1,F,сотрудник,0,95856,6,0,2
9,0,-2188,41,1,0,M,сотрудник,0,144425,7,4,0


Категоризируем уровень дохода клиентов. Создадим словарь с идентификаторами уровня дохода и описаниями категорий:
- В таблице `customer_solvency` создадим столбец `total_income_cat_name`, куда запишем категории уровня доходов, созданные с помощью метода `qcut()`.  
- Далее создадим словарь `total_income_dict` c названиями и идентификаторами полученных ранее категорий.
- С помощь метода `merge()` соотнесем названия и идентификаторы категорий доходов в основной таблице
- Удалим названия категорий доходов из основной таблицы, оставив только иденнтификаторы (для наглядности и экономии места, занимаемого основной таблицей.)

In [28]:
# с помощью метода `qcut` создадим 6 категорий доходов с одинаковым колличеством клиентов в каждой группе. 
# Параметр `precision=0` необходим для целочисленного разделения на группы.
customer_solvency['total_income_cat_name'] = pd.qcut(customer_solvency['total_income'], q=6, precision=0)
# Выделим уникальные группы с помощью метода value_counts() и запишем их в отдельную таблицу
total_income_dict = customer_solvency['total_income_cat_name'].value_counts().reset_index()
# Нет необходимости хранить колличество записей для каждой группы, удалим эти данный с помощью метода `drop()`
total_income_dict = total_income_dict.drop('total_income_cat_name', axis=1)
# Переименуем столбец с названиями групп `total_income_cat_name`
total_income_dict.columns = ['total_income_cat_name']
# Отсортируем группы по возрастанию доходов с помощью метода `sort_values` и сбросим индексы в таблице.
total_income_dict = total_income_dict.sort_values('total_income_cat_name').reset_index(drop = True)
# Сброшенные индексы можно использовать как идентификаторы категорий доходов клиентов.
# Запишем идентификаторы в столбец `total_income_cat_id`
total_income_dict['total_income_cat_id'] = total_income_dict.index
display(total_income_dict)

,total_income_cat_name,total_income_cat_id
0,"(20666.0, 92141.0]",0
1,"(92141.0, 122618.0]",1
2,"(122618.0, 141139.0]",2
3,"(141139.0, 175340.0]",3
4,"(175340.0, 228859.0]",4
5,"(228859.0, 2265604.0]",5


In [29]:
# С помощь метода merge() соотнесём названия и идентификаторы категорий доходов в основной таблице
customer_solvency = customer_solvency.merge(total_income_dict, on='total_income_cat_name', how='left')
# Удалим названия категорий доходов из основной таблицы с помощью метода `drop()`
customer_solvency = customer_solvency.drop('total_income_cat_name', axis=1)
display(customer_solvency.head(10))

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose_id,purpose_cat_id,children_cat_id,total_income_cat_id
0,1,-8437,42,0,0,F,сотрудник,0,253875,0,4,1,5
1,1,-4024,36,1,0,F,сотрудник,0,112080,1,2,1,1
2,0,-5623,33,1,0,M,сотрудник,0,145885,0,4,0,3
3,3,-4124,32,1,0,M,сотрудник,0,267628,2,3,3,5
4,0,340266,53,1,1,F,пенсионер,0,158616,3,0,0,3
5,0,-926,27,0,1,M,компаньон,0,255763,0,4,0,5
6,0,-2879,43,0,0,F,компаньон,0,240525,4,4,0,5
7,0,-152,50,1,0,M,сотрудник,0,135823,5,3,0,2
8,2,-6929,35,0,1,F,сотрудник,0,95856,6,0,2,1
9,0,-2188,41,1,0,M,сотрудник,0,144425,7,4,0,3


### Вывод

При категоризации данных были выделены следующие словари:

- Цели кредита (словарь `purposes`)
- Ежемесячный доход (словарь `total_income_dict`)
- Уровень образования клиента (словарь `education_dict`)
- Семейное положение (словарь `family_status_dict`)
- Колличество детей (словарь `children_dict`)  
  
Вся информация о категориях была вынесена в словари, в основной таблице оставлены идентификааторы категорий. Данные преобразования были сделаны для сокращения места, занимаемого основной таблицей, а также для наглядности и удобства работы.

<a id='секция3'></a>
## Шаг 3. Исследование данных

<a id='секция3.1'></a>
### 3.1 Зависимость между наличием детей и возвратом кредита в срок

Для ответа на вопрос составим новую таблицу `children_debt` с данными о задолженностях по кредиту `debt` из таблицы `customer_solvency` и информацией о колличестве детей `children_cat_id`, `children_cat_name` из словаря `children_dict`. 
Далее для наглядности результаты выведем в сводную таблицу.  
Также рассчитаем отношение колличества клиентов с задолженностями к общему колличеству клиентов по каждой группе.

In [30]:
children_debt = customer_solvency.merge(children_dict, on='children_cat_id', how='left')[['children_cat_id', 
                                                                             'children_cat_name', 
                                                                             'debt']]
# сводную таблицу получим методом pivot_table(). 
children_debt_pivot = children_debt.pivot_table(index = 'children_cat_name', columns = 'debt', 
                                                values = 'children_cat_id', aggfunc = 'count')
# найдем в каждой группе долю клиентов с задоленностями относительно 
# общего числа клиентов в данной группе
children_debt_pivot['rate'] = children_debt_pivot[1] / (children_debt_pivot[0] + children_debt_pivot[1])
display(children_debt_pivot)

debt,0,1,rate
children_cat_name,,,
Двое детей,1858,194,0.094542
Многодетная семья,417,39,0.085526
Нет детей,13027,1063,0.075444
Один ребенок,4364,444,0.092346
Статус неизвестен,46,1,0.021277


### Вывод

- Минимальная доля клиентов с задолженностями среди группы без детей (7.5%). 
- Среди клиентов с одним и двумя детьми данные мало отличаются, примерно 9.5% клиентов имеют задолженности.
- Среди многодетных семей доля клиентов с задолженностями составляет около 8.5%. Но в данной группе сравнительно небольшая выборка и необходимо дальнейшее иследование относительно достоверности результатов.
- В целом доля задолженностей по кредитам среди клиентов не сильно меняется от колличества детей в семье (в пределах 1-2%).

<a id='секция3.2'></a>
### 3.2 Зависимость между семейным положением и возвратом кредита в срок

Для ответа на вопрос составим новую таблицу `family_status_debt` с данными о задолженностях по кредиту `debt` из таблицы `customer_solvency` и информацией о семейном статусе `family_status_id`, `family_status` из словаря `family_status_dict`.  
Далее для наглядности результаты выведем в сводную таблицу.  
Также рассчитаем отношение колличества клиентов с задолженностями к общему колличеству клиентов по каждой группе.

In [31]:
family_status_debt = customer_solvency.merge(family_status_dict, 
                                on='family_status_id', how='left')[['family_status_id',
                                                                   'family_status',
                                                                   'debt']]
# сводную таблицу получим методом pivot_table(). 
family_status_pivot = family_status_debt.pivot_table(index = 'family_status', columns = 'debt',
                                                     values = 'family_status_id', aggfunc = 'count')
# найдем в каждой группе долю клиентов с задоленностями относительно 
# общего числа клиентов в данной группе
family_status_pivot['rate'] = family_status_pivot[1] / (family_status_pivot[0] + family_status_pivot[1])
display(family_status_pivot)

debt,0,1,rate
family_status,,,
в разводе,1110,85,0.071130
вдовец / вдова,896,63,0.065693
гражданский брак,3762,388,0.093494
женат / замужем,11408,931,0.075452
не женат / не замужем,2536,274,0.097509


### Вывод

- Минимальная доля клиентов с задолженностями среди группы `вдовец / вдова` (6.5%). 
- Среди клиентов групп `не женат / не замужем` и `гражданский брак` данные мало отличаются, примерно 9.5% клиентов имеют задолженности. Это максимальные показатели среди всех групп
- Среди клиентов группы `вдовец / вдова` доля задолженностей минимальна и составляет около 6.5%. Но в данной группе сравнительно небольшая выборка и необходимо дальнейшее иследование относительно достоверности результатов.
- Среди клиентов в категориях `женат / замужем` и `в разводе` доля задолженностей примерно одинакова и составляет 7-7.5%. Но выборка клиентов в разводе сравнительно неболшая, и данные могут быть недостоверны.
- Судя по полученным данным, семейные клиенты имеют меньше задолженностей по кредитам, чем неженатые/незамужние клиенты или в гражданском браке. Возможно, семейные пары более тщательно планируют свой бюджет и серьезнее относятся к своим обязательствам.
- В целом доля задолженностей по кредитам среди клиентов не сильно меняется от семейного положения (в пределах 3%).

<a id='секция3.3'></a>
### 3.3 Зависимость между уровнем дохода и возвратом кредита в срок

Для ответа на вопрос составим новую таблицу `total_income_debt` с данными о задолженностях по кредиту `debt` из таблицы `customer_solvency`, а также информацией о ежемесячных доходах `total_income_cat_id` и `total_income_cat_name` из словаря `total_income_dict`.  
Далее для наглядности результаты выведем в сводную таблицу.  
Также рассчитаем отношение колличества клиентов с задолженностями к общему колличеству клиентов по каждой группе.  

In [32]:
total_income_debt = customer_solvency.merge(total_income_dict, 
                                on='total_income_cat_id', how='left')[['total_income_cat_id',
                                                                   'total_income_cat_name',
                                                                   'debt']]
# сводную таблицу получим методом pivot_table(). 
total_income_pivot = total_income_debt.pivot_table(index = 'total_income_cat_name', columns = 'debt',
                                                     values = 'total_income_cat_id', aggfunc = 'count')
# найдем в каждой группе долю клиентов с задоленностями относительно 
# общего числа клиентов в данной группе
total_income_pivot['rate'] = total_income_pivot[1] / (total_income_pivot[0] + total_income_pivot[1])
display(total_income_pivot.sort_values('rate'))

debt,0,1,rate
total_income_cat_name,,,
"(228859.0, 2265604.0]",3326,250,0.069911
"(175340.0, 228859.0]",2917,252,0.079520
"(20666.0, 92141.0]",3291,285,0.079698
"(92141.0, 122618.0]",3277,298,0.083357
"(141139.0, 175340.0]",3642,339,0.085154
"(122618.0, 141139.0]",3259,317,0.088647


### Вывод

- Клиенты с высоким уровнем доходов (> 230к) в целом отдают кредиты в срок чаще (7% клиентов с задолженностями). 
- Клиенты с более низкими доходами (до 90к), а также с доходами в 175к-230к в среднем имеют задолженности в 8% случаев. Это может быть связано с нехваткой финансов или проблемами с планированием бюджета.
- Клиенты со средними доходами (90к - 175к) имеют задолженности в 8-9% случаев. Это может быть связано с проблемами в планировании бюджета (большие сумммы кредитов и маленькие сроки могут приводить к высоким ежемесячным выплатам, с которыми клиенты не всегда справляются).
- В целом данные не сильно отличаются с изменением уровня доходов (в пределах 2%).

<a id='секция3.4'></a>
### 3.4 Влияние целей кредита на его возврат в срок

Для ответа на вопрос составим новую таблицу `purposes_debt` с данными о задолженностях по кредиту `debt` из таблицы `customer_solvency`, а также информацией о ежемесячных доходах `purpose_category` и `purpose_cat_id` из словаря `purposes`.  
Далее для наглядности результаты выведем в сводную таблицу.  
Также рассчитаем отношение колличества клиентов с задолженностями к общему колличеству клиентов по каждой группе.  

In [33]:
purposes_debt = customer_solvency.merge(purposes,
                           on='purpose_cat_id', how='left')[['purpose_cat_id',
                                                             'purpose_category',
                                                             'debt']]
# сводную таблицу получим методом pivot_table(). 
purposes_debt_pivot = purposes_debt.pivot_table(index = 'purpose_category', columns = 'debt',
                                                     values = 'purpose_cat_id', aggfunc = 'count')
# найдем в каждой группе долю клиентов с задоленностями относительно 
# общего числа клиентов в данной группе
purposes_debt_pivot['rate'] = purposes_debt_pivot[1] / (purposes_debt_pivot[0] + purposes_debt_pivot[1])
display(purposes_debt_pivot.sort_values('rate'))

debt,0,1,rate
purpose_category,,,
ремонт,572,35,0.057661
жилье/недвижимость,151296,11952,0.073214
свадьба,6414,558,0.080034
образование,32787,3330,0.092200
автомобиль,35127,3627,0.093590


### Вывод

- Кредиты на ремонт квартиры в среднем задерживают в 5.7% случаев. Но выборка в этой категории относительно мала, сложно судить о достоверности результатов.
- Больше всего кредитов берут на покупку и другие операции с недвижимостью. Задолженности возникают у 7.3% клиентов.
- Кредиты на образование и покупку автомобиля задерживают в 9-9.5% случаев (на 2% больше чем кредиты, связанные с недвижимостью). На данном этапе сложно определить причины такой разницы.
- В целом данные по задолженностям не сильно меняются от целей, на котоые взяты кредиты (в пределах 3 - 3.5%).

<a id='секция4'></a>
## Шаг 4. Общий вывод

1) В Работе были проанализированы данные о статистике платежеспособности клиентов.
Были обнаружены пропущенные данные в одинаковых записях столбцов `days_employed` и `total_income`. Возможные причины - сбой системы при загрузке/выгрузке данных из базы.  
Также в столбце `days_employed` обнаружены отрицательные и очень высокие значения. Причинами могли стать ошибки при заполнени данных, а также сбои при работе с базой данных. Точные причины некорректных данных на назвать сложно. Необходимо дальнейшее исследование ошибок с сотрудниками банка.    
Посколько данные `days_employed` в дальнейших исследованиях не участвуют, пропущенные данные были заменены нулями для наглядности.  
Данные в столбце `total_income` были заменены медианнными значениями групп с соответствующим уровнем образования клиентов.  
Так же были обнаружены 46 записей с отрицательным значением колличества детей. Было сделано предположение, что колличество детей у данных клиентов неизвестно. Данные сохранены в отдельной группе.  
2) После преобразования строковых данных была обнаружена 71 дублированная строка. Поскольку в данных нет уникальных идентификаторов клиентов, невозможно сделать точно сказать, дублированы ли данные одних и тех же клиентов или информация по разным клиентам совпала случайным образом.  
Дублированные записи были удалены в учебных целях.  
3) Среди целей на получение кредита было выделено 5 основных групп: `свадьба`, `ремонт`, `автомобиль`, `образование`, `жилье/недвижимость`. Лемматизация целей проводилась с использованием библиотеки `pymystem3`.  
4) При категоризации данных были выделены словари с информацией:
    - Цели кредита (словарь `purposes`)
    - Ежемесячный доход (словарь `total_income_dict`)
    - Уровень образования клиента (словарь `education_dict`)
    - Семейное положение (словарь `family_status_dict`)
    - Колличество детей (словарь `children_dict`)  
Вся информация о категориях была вынесена в словари, в основной таблице оставлены идентификааторы категорий. Данные преобразования были сделаны для сокращения места, занимаемого основной таблицей, а также для наглядности и удобства работы.  
5) Были проанализированы задолженности клиентов по кредитам в зависимости от количества детей в семье, семейного положения, уровня месячного дохода и цели кредита.  
Данные выведены в сводные таблицы. Дополнительно рассчитаны доли клиентов с задолженностями для каждой категории.  
В некоторых  категориях выборки оказались сравнительно малы, что не позволяет делать однозначные выводы о результатах исследований. Возможные пути решения проблемы - наращивать базу данных.  
По всем категориям доли клиентов с задолженностями не превышала 10%, а различия значений между отдельными группами разделения не превышали 3.5%.  